In [1]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&

In [2]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package w3m.
(Reading database ... 122354 files and directories currently installed.)
Preparing to unpack .../w3m_0.5.3-37ubuntu0.1_amd64.deb ...
Unpacking w3m (0.5.3-37ubuntu0.1) ...
Setting up w3m (0.5.3-37ubuntu0.1) ...
Processing triggers for man-db (2.9.1-1) ...
Processing triggers for mime-support (3.64ubuntu1) ...
/content
/content/drive
/content
/
Access token retrieved correctly.


In [16]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt


In [17]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/MIMIA/Covid19-dataset-HistEq-BilateralFilter/train'
valid_path = '/content/drive/MyDrive/MIMIA/Covid19-dataset-HistEq-BilateralFilter/test'

In [18]:
# Import the VGG16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [19]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False


In [20]:
# our layers - you can add more if you want
x = Flatten()(vgg16.output)

In [21]:
prediction = Dense(3, activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [22]:
# view the structure of the model
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [23]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [24]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [25]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/MIMIA/Covid19-dataset-HistEq-BilateralFilter/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 251 images belonging to 3 classes.


In [26]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/MIMIA/Covid19-dataset-HistEq-BilateralFilter/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 66 images belonging to 3 classes.


In [27]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-27-6ec5a35725ca>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/5
8/8 [==============================] - 16s 2s/step - loss: 1.8700 - accuracy: 0.4781 - val_loss: 1.3281 - val_accuracy: 0.6515
Epoch 2/5
8/8 [==============================] - 14s 2s/step - loss: 0.4593 - accuracy: 0.8287 - val_loss: 0.2877 - val_accuracy: 0.9242
Epoch 3/5
8/8 [==============================] - 14s 2s/step - loss: 0.2128 - accuracy: 0.9283 - val_loss: 0.3133 - val_accuracy: 0.8030
Epoch 4/5
8/8 [==============================] - 14s 2s/step - loss: 0.1934 - accuracy: 0.9283 - val_loss: 0.1652 - val_accuracy: 0.9242
Epoch 5/5
8/8 [==============================] - 15s 2s/step - loss: 0.1384 - accuracy: 0.9522 - val_loss: 0.1965 - val_accuracy: 0.9242


In [28]:
# fit the model
# Run the cell. It will take some time to execute
r1 = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-28-b4bf0cc3fc21>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r1 = model.fit_generator(


Epoch 1/10
8/8 [==============================] - 15s 2s/step - loss: 0.1137 - accuracy: 0.9522 - val_loss: 0.1115 - val_accuracy: 0.9848
Epoch 2/10
8/8 [==============================] - 15s 2s/step - loss: 0.0776 - accuracy: 0.9801 - val_loss: 0.1150 - val_accuracy: 0.9697
Epoch 3/10
8/8 [==============================] - 14s 2s/step - loss: 0.0887 - accuracy: 0.9721 - val_loss: 0.0925 - val_accuracy: 0.9848
Epoch 4/10
8/8 [==============================] - 14s 2s/step - loss: 0.0633 - accuracy: 0.9841 - val_loss: 0.0944 - val_accuracy: 1.0000
Epoch 5/10
8/8 [==============================] - 14s 2s/step - loss: 0.0552 - accuracy: 0.9880 - val_loss: 0.0845 - val_accuracy: 0.9848
Epoch 6/10
8/8 [==============================] - 14s 2s/step - loss: 0.0461 - accuracy: 0.9960 - val_loss: 0.0835 - val_accuracy: 0.9848
Epoch 7/10
8/8 [==============================] - 14s 2s/step - loss: 0.0510 - accuracy: 0.9880 - val_loss: 0.0795 - val_accuracy: 0.9848
Epoch 8/10
8/8 [==================